# Coursera Capstone Project
## Find the best location to open a pharmacy

import all needed Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge geocoder --yes
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
# Create dataframe with all Postal Codes of Dammam City
dmm_df = pd.DataFrame(np.arange(32211,32781,1),columns=['Postal_Code'])
dmm_df.shape
dmm_df.head()

,Postal_Code
0,32211
1,32212
2,32213
3,32214
4,32215


In [3]:
# define a function to get coordinates
def get_latlng(Postal_Code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Dammam, Saudi Arabia'.format(Postal_Code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [4]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(Postal_Code) for Postal_Code in dmm_df["Postal_Code"].tolist() ]

In [5]:
coords

[[26.49982574300003, 50.19982269700006],
 [26.437087863000045, 50.18724292200005],
 [26.421793878000074, 50.16912676600003],
 [26.41058399900004, 50.18493592800007],
 [26.440410000000043, 50.06027000000006],
 [26.440410000000043, 50.06027000000006],
 [26.440410000000043, 50.06027000000006],
 [26.440410000000043, 50.06027000000006],
 [26.440410000000043, 50.06027000000006],
 [26.440410000000043, 50.06027000000006],
 [26.412785000000042, 50.16590104000005],
 [26.401352321000047, 50.17443000000003],
 [26.402795541000046, 50.16720000000004],
 [26.405882254000062, 50.15968000000004],
 [26.39182364100003, 50.162765000000036],
 [26.395650660000058, 50.15684500000003],
 [26.440410000000043, 50.06027000000006],
 [26.440410000000043, 50.06027000000006],
 [26.440410000000043, 50.06027000000006],
 [26.440410000000043, 50.06027000000006],
 [26.40814358800003, 50.15023971200003],
 [26.420577104000074, 50.139734676000046],
 [26.428377173000058, 50.123357018000036],
 [26.395489588000032, 50.1398650000

In [6]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [7]:
# merge the coordinates into the original dataframe
dmm_df['Latitude'] = df_coords['Latitude']
dmm_df['Longitude'] = df_coords['Longitude']

In [50]:
# check the neighborhoods and the coordinates and drop the duplicates
print(dmm_df.shape)
dmm_df.drop_duplicates(subset=['Latitude','Longitude'],inplace=True)
print(dmm_df.shape)
dmm_df

(156, 3)
(156, 3)


,Postal_Code,Latitude,Longitude
0,32211,26.499826,50.199823
1,32212,26.437088,50.187243
2,32213,26.421794,50.169127
3,32214,26.410584,50.184936
4,32215,26.440410,50.060270
10,32221,26.412785,50.165901
11,32222,26.401352,50.174430
12,32223,26.402796,50.167200
13,32224,26.405882,50.159680
14,32225,26.391824,50.162765


In [9]:
# save the DataFrame as CSV file
dmm_df.to_csv("dmm_df.csv", index=False)

In [10]:
# get the coordinates of Dammam
address = 'Dammam, Saudi Arabia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dammam ,Saudi Arabia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dammam ,Saudi Arabia 26.4367824, 50.1039991.


In [32]:
# create map of Dammam using latitude and longitude values
map_dmm = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(dmm_df['Latitude'], dmm_df['Longitude'], dmm_df['Postal_Code']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dmm)  
    
map_dmm

In [12]:
# save the map as HTML file
map_dmm.save('map_dmm.html')

In [13]:

# define Foursquare Credentials and Version
CLIENT_ID = '33ZVJSGFOD2CQQYDFBRJS3DLUW52CXLLA4YABWFG0V4KLQLZ' # your Foursquare ID
CLIENT_SECRET = 'NRMX20GYRNOE4JCUFY5TMQ02X12BK2DMBY3IXTA54ELN5ABS' # your Foursquare Secret
VERSION = '20190801' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 33ZVJSGFOD2CQQYDFBRJS3DLUW52CXLLA4YABWFG0V4KLQLZ
CLIENT_SECRET:NRMX20GYRNOE4JCUFY5TMQ02X12BK2DMBY3IXTA54ELN5ABS


In [14]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(dmm_df['Latitude'], dmm_df['Longitude'], dmm_df['Postal_Code']):
    print(neighborhood,lat, long)
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT,)
    
    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

32211 26.49982574300003 50.19982269700006
32212 26.437087863000045 50.18724292200005
32213 26.421793878000074 50.16912676600003
32214 26.41058399900004 50.18493592800007
32215 26.440410000000043 50.06027000000006
32221 26.412785000000042 50.16590104000005
32222 26.401352321000047 50.17443000000003
32223 26.402795541000046 50.16720000000004
32224 26.405882254000062 50.15968000000004
32225 26.39182364100003 50.162765000000036
32226 26.395650660000058 50.15684500000003
32231 26.40814358800003 50.15023971200003
32232 26.420577104000074 50.139734676000046
32233 26.428377173000058 50.123357018000036
32234 26.395489588000032 50.13986500000004
32235 26.39423789800003 50.12380500000006
32236 26.409998061000067 50.118025000000046
32241 26.434224536000045 50.112880000000075
32242 26.435100142000067 50.10239500000006
32243 26.425940000000026 50.09094761400007
32244 26.438130000000058 50.08995875000005
32245 26.422792449000042 50.077825000000075
32246 26.434415000000058 50.075590684000076
32247 26.

In [15]:
# note

In [33]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Postal_Code', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6307, 7)


,Postal_Code,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,32211,26.499826,50.199823,Ship Side,26.507674,50.198183,Port
1,32211,26.499826,50.199823,IPS Container&RORO Terminal,26.508555,50.201615,Port
2,32211,26.499826,50.199823,SGPC Group,26.505077,50.188356,Cruise
3,32211,26.499826,50.199823,مبنى اكاديمية الضباط البحريين,26.487013,50.199271,Port
4,32211,26.499826,50.199823,الميناء التجاري,26.497673,50.215691,Boat or Ferry


In [34]:
venues_df.groupby(["Postal_Code"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Postal_Code,,,,,,
32211,6,6,6,6,6,6
32212,7,7,7,7,7,7
32213,11,11,11,11,11,11
32214,17,17,17,17,17,17
32215,100,100,100,100,100,100
32221,15,15,15,15,15,15
32222,38,38,38,38,38,38
32223,35,35,35,35,35,35
32224,27,27,27,27,27,27


In [17]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 237 uniques categories.


In [20]:
# print out the list of categories
venues_df['VenueCategory'].unique()[0:40]

array(['Port', 'Cruise', 'Boat or Ferry', 'Shipping Store',
       'Jewelry Store', 'Fishing Spot', 'Italian Restaurant',
       'Intersection', 'Seafood Restaurant', 'Coffee Shop', 'Board Shop',
       'Café', 'Fast Food Restaurant', 'Department Store', 'Market',
       'Motorcycle Shop', 'Park', 'Soccer Field', 'Auto Garage',
       'Donut Shop', 'Gym / Fitness Center', 'Pharmacy', 'Restaurant',
       'Beach', 'College Gym', 'Athletics & Sports', 'Bakery',
       'Breakfast Spot', 'Convenience Store', 'Candy Store',
       'Cupcake Shop', 'Fried Chicken Joint', 'Falafel Restaurant',
       'Discount Store', 'Fish Market', 'Ice Cream Shop', 'Pizza Place',
       'Comedy Club', 'Burger Joint', 'Grocery Store'], dtype=object)

In [21]:
# check if the results contain "Pharmacy"
"Pharmacy" in venues_df['VenueCategory'].unique()

True

In [35]:
# one hot encoding
dmm_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dmm_onehot['Postal_Code'] = venues_df['Postal_Code'] 

# move neighborhood column to the first column
fixed_columns = [dmm_onehot.columns[-1]] + list(dmm_onehot.columns[:-1])
dmm_onehot = dmm_onehot[fixed_columns]

print(dmm_onehot.shape)
dmm_onehot.head()

(6307, 238)


,Postal_Code,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Big Box Store,Bike Trail,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Business Service,Cable Car,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Carpet Store,Castle,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Coffee Shop,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Country Dance Club,Cruise,Cuban Restaurant,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Donut Shop,Dutch Restaurant,Duty-free Shop,Egyptian Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fire Station,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gift Shop,Go Kart Track,Gourmet Shop,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hunting Supply,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Insurance Office,Intersection,Iraqi Restaurant,Italian Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Kosher Restaurant,Laundry Service,Lebanese Restaurant,Lounge,Luggage Store,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Night Market,Nightclub,Noodle House,Office,Optical Shop,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Palace,Park,Pastry Shop,Perfume Shop,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Port,Post Office,Print Shop,RV Park,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shopping Mall,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Speakeasy,Sporting Goods Shop,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Summer Camp,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Tea Room,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Trail,Train Station,Travel Lounge,Tunnel,Turkish Restaurant,Video Game Store,Warehouse Store,Water Park,Waterfront,Yemeni Restaurant
0,32211,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,32211,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [36]:
dmm_grouped = dmm_onehot.groupby(["Postal_Code"]).mean().reset_index()

print(dmm_grouped.shape)
dmm_grouped

(150, 238)


,Postal_Code,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Big Box Store,Bike Trail,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Business Service,Cable Car,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Carpet Store,Castle,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Coffee Shop,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Country Dance Club,Cruise,Cuban Restaurant,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Donut Shop,Dutch Restaurant,Duty-free Shop,Egyptian Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fire Station,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gift Shop,Go Kart Track,Gourmet Shop,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hunting Supply,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Insurance Office,Intersection,Iraqi Restaurant,Italian Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Kosher Restaurant,Laundry Service,Lebanese Restaurant,Lounge,Luggage Store,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Night Market,Nightclub,Noodle House,Office,Optical Shop,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Palace,Park,Pastry Shop,Perfume Shop,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Port,Post Office,Print Shop,RV Park,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shopping Mall,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Speakeasy,Sporting Goods Shop,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Summer Camp,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Tea Room,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Trail,Train Station,Travel Lounge,Tunnel,Turkish Restaurant,Video Game Store,Warehouse Store,Water Park,Waterfront,Yemeni Restaurant
0,32211,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.166667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.166667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [24]:
len(dmm_grouped[dmm_grouped["Pharmacy"] > 0])

67

In [37]:
dmm_mall = dmm_grouped[["Postal_Code","Pharmacy"]]

In [38]:
dmm_mall.head()

,Postal_Code,Pharmacy
0,32211,0.000000
1,32212,0.000000
2,32213,0.000000
3,32214,0.117647
4,32215,0.020000


In [39]:
# set number of clusters
kclusters = 3

dmm_clustering = dmm_mall.drop(["Postal_Code"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dmm_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 2, 1, 0, 1, 1, 1, 1], dtype=int32)

In [40]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
dmm_merged = dmm_mall.copy()

# add clustering labels
dmm_merged["Cluster Labels"] = kmeans.labels_

In [41]:
#dmm_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
dmm_merged.head()

,Postal_Code,Pharmacy,Cluster Labels
0,32211,0.000000,0
1,32212,0.000000,0
2,32213,0.000000,0
3,32214,0.117647,2
4,32215,0.020000,1


In [42]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dmm_merged = dmm_merged.join(dmm_df.set_index("Postal_Code"), on="Postal_Code")

print(dmm_merged.shape)
dmm_merged.head() # check the last columns!

(150, 5)


,Postal_Code,Pharmacy,Cluster Labels,Latitude,Longitude
0,32211,0.000000,0,26.499826,50.199823
1,32212,0.000000,0,26.437088,50.187243
2,32213,0.000000,0,26.421794,50.169127
3,32214,0.117647,2,26.410584,50.184936
4,32215,0.020000,1,26.440410,50.060270


In [43]:
# sort the results by Cluster Labels
print(dmm_merged.shape)
dmm_merged.sort_values(["Cluster Labels"], inplace=True)
dmm_merged

(150, 5)


,Postal_Code,Pharmacy,Cluster Labels,Latitude,Longitude
0,32211,0.000000,0,26.499826,50.199823
114,32513,0.000000,0,26.478286,49.981896
113,32512,0.000000,0,26.454807,49.995644
106,32446,0.000000,0,26.441115,50.019029
105,32445,0.012658,0,26.428149,50.020432
104,32444,0.012987,0,26.432062,50.031185
95,32432,0.010000,0,26.447227,50.058345
92,32426,0.010000,0,26.446870,50.075451
91,32425,0.010000,0,26.446255,50.088528
87,32421,0.000000,0,26.478186,50.091000


In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dmm_merged['Latitude'], dmm_merged['Longitude'], dmm_merged['Postal_Code'], dmm_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [52]:
dmm_merged[dmm_merged['Cluster Labels']==2]

,Postal_Code,Pharmacy,Cluster Labels,Latitude,Longitude
12,32232,0.071429,2,26.420577,50.139735
45,32313,0.142857,2,26.387065,50.002582
99,32436,0.082192,2,26.458370,50.040940
3,32214,0.117647,2,26.410584,50.184936
52,32324,0.066667,2,26.372155,49.971646
